# Web scraping - Peruvian Football News (Peru21)

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from html2text import html2text
import re
from nltk import word_tokenize
import datetime
import locale
import time
import requests
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time

In [ ]:
def check_text(text_new):
    text_new = re.sub(r'[^a-zA-Z0-9ÑñáéíóúÁÉÍÓÚ!¡:¿?\-,;.\s]', '', text_new).strip()
    return text_new

In [5]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

date_n = datetime.datetime.strptime("2023-11-03", "%Y-%m-%d")
date_u = datetime.datetime.strptime("2023-01-01", "%Y-%m-%d")

total_news = 0
links = list()

while True:

    try:
        day = date_n.strftime("%d")
        month = date_n.strftime("%m")
        year = date_n.strftime("%Y")

        url = "https://peru21.pe/archivo/todas/{}-{}-{}/".format(year, month, day)

        print(date_n.strftime("Scraping the news from %d/%m/%Y"))

        driver.get(url)

        for i in range(4):
            # scroll to the bottom of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

        elem_hrefs = driver.find_elements(by=By.XPATH, value='//a[contains(text(), "Fútbol peruano")]/parent::div/h2/a')
        if elem_hrefs:
            print("Found {} peruvian football news".format(len(elem_hrefs)))
            total_news = total_news + len(elem_hrefs)
        else:
            print("Not found peruvian football")

        for ele in elem_hrefs:
            links.append(ele.get_attribute("href"))

        date_n = date_n - datetime.timedelta(days=1)

        print("Correct scraping! Total news: {}".format(total_news))
        time.sleep(2)

        if date_n == date_u:
            break
        else:
            continue

    except Exception as e:
        print(e)
        print("Incorrect scraping!")

Scraping the news from 03/11/2023
Not found peruvian football
Correct scraping! Total news: 0
Scraping the news from 02/11/2023
Not found peruvian football
Correct scraping! Total news: 0
Scraping the news from 01/11/2023
Found 2 peruvian football news
Correct scraping! Total news: 2
Scraping the news from 31/10/2023
Found 1 peruvian football news
Correct scraping! Total news: 3
Scraping the news from 30/10/2023
Found 1 peruvian football news
Correct scraping! Total news: 4
Scraping the news from 29/10/2023
Found 8 peruvian football news
Correct scraping! Total news: 12
Scraping the news from 28/10/2023
Found 1 peruvian football news
Correct scraping! Total news: 13
Scraping the news from 27/10/2023
Not found peruvian football
Correct scraping! Total news: 13
Scraping the news from 26/10/2023
Not found peruvian football
Correct scraping! Total news: 13
Scraping the news from 25/10/2023
Not found peruvian football
Correct scraping! Total news: 13
Scraping the news from 24/10/2023
Not fo

In [40]:
df_football_news_2 = pd.DataFrame()

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

for link in links:

    driver.get(link)

    time.sleep(5)

    foot_new = driver.find_elements(by=By.XPATH, value='//p')
    ele_title = driver.find_element(by=By.XPATH, value='//h1')
    ele_subtitle = driver.find_element(by=By.XPATH, value="//h2[contains(@class, 'sht__summary')]")

    full_text_new = ''
    long_new = len(foot_new)
    for idx, ele in enumerate(foot_new):
        if idx not in [0, long_new-1]:
            text_new = check_text(ele.text)
            if text_new != '':
                full_text_new = full_text_new + " " + text_new
    full_text_new = full_text_new.strip()

    football_title = ele_title.text
    football_title = check_text(football_title)


    football_subtitle = ele_subtitle.text
    football_subtitle = check_text(football_subtitle)

    df_football_news_c = pd.DataFrame({"Title": [football_title], "Subtitle": [football_subtitle], "Text": [full_text_new]})
    df_football_news_2 = pd.concat([df_football_news_2, df_football_news_c], axis=0)

    print("Correct scraping: {}".format(link))

driver.close()


df_football_news_2.reset_index(drop=True, inplace=True)

df_football_news_2['Text'] = df_football_news_2['Text'].apply(lambda x: x.replace("\n", ""))

Correct scraping: https://peru21.pe/deportes/futbol-peruano/pide-seleccion-oliver-sonne-anoto-en-la-goleada-del-silkeborg-por-la-copa-de-dinamarca-peru-oliver-sonne-futbol-internacional-peruanos-en-el-extranjero-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/alianza-lima-leao-butron-casi-se-fue-a-las-manos-contra-hincha-en-el-aeropuerto-video-libertadores-inclusiva-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/se-pierde-la-primera-final-williams-riveros-hizo-trabajos-diferenciados-y-causa-preocupacion-peru-universitario-alianza-lima-entrenamiento-final-liga-1-williams-riveros-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/el-kaiser-esta-listo-carlos-zambrano-se-recupero-y-llegara-a-las-finales-ante-universitario-carlos-zambrano-alianza-lima-universitario-final-liga-1-torneo-clausura-clasico-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/alianza-lima-vs-universitario-hernan-barcos-seria-lindo-ser-tr

In [53]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

url = "https://peru21.pe/deportes/futbol-peruano/"

driver.get(url)

df_football_news = pd.DataFrame()

elem_hrefs = driver.find_elements(by=By.XPATH, value='//a[contains(text(), "Fútbol peruano")]/parent::div/h2/a')
links = [ele.get_attribute("href") for ele in elem_hrefs]

for link in links:

    driver.get(link)

    time.sleep(5)

    foot_new = driver.find_elements(by=By.XPATH, value='//p')
    ele_title = driver.find_element(by=By.XPATH, value='//h1')
    ele_subtitle = driver.find_element(by=By.XPATH, value="//h2[contains(@class, 'sht__summary')]")

    full_text_new = ''
    long_new = len(foot_new)
    for idx, ele in enumerate(foot_new):
        if idx not in [0, long_new-1]:
            text_new = check_text(ele.text)
            if text_new != '':
                full_text_new = full_text_new + " " + text_new
    full_text_new = full_text_new.strip()

    football_title = ele_title.text
    football_title = check_text(football_title)


    football_subtitle = ele_subtitle.text
    football_subtitle = check_text(football_subtitle)

    df_football_news_c = pd.DataFrame({"Title": [football_title], "Subtitle": [football_subtitle], "Text": [full_text_new]})
    df_football_news = pd.concat([df_football_news, df_football_news_c], axis=0)

    print("Correct scraping: {}".format(link))

driver.close()


df_football_news.reset_index(drop=True, inplace=True)

Correct scraping: https://peru21.pe/deportes/futbol-peruano/premio-al-esfuerzo-piero-quispe-fue-elegido-como-el-mejor-jugador-de-la-liga-1-2023-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/a-festejar-crema-hoy-es-la-premiacion-de-universitario-como-campeon-nacional-2023-peru-universitario-campeon-futbol-peruano-liga-1-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/universitario-edison-flores-tienen-que-venir-mas-extranjeros-como-hernan-barcos-a-la-liga-1-titulo-de-la-u-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/premiacion-liga-1-quien-fue-el-mejor-jugador-del-futbol-peruano-este-ano-piero-quispe-universitario-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/universitario-jean-ferrari-si-queremos-a-yoshimar-yotun-video-refuerzos-2024-sporting-cristal-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/universitario-altas-y-bajas-para-el-proximo-ano-centenario-u-plantel-2024-refue

In [54]:
df_football_news['Text'] = df_football_news['Text'].apply(lambda x: x.replace("\n", ""))

In [55]:
df_football_news.head(5)

,Title,Subtitle,Text
0,Premio al esfuerzo: Piero Quispe fue elegido c...,El mediocampista de Universitario fue pieza fu...,"Luego de 10 años de sequía, Universitario form..."
1,A festejar crema: así fue la premiación de Uni...,"En el Monumental de Ate, los merengues serán p...",El equipo crema se dispone a dar la vuelta olí...
2,¿Se disculpó con Barcos? Flores: Tienen que ve...,Después de imitarlo en la celebración por el t...,En medio de las celebraciones por su nuevo tít...
3,Liga 1: ¿Quién fue el mejor jugador del fútbol...,Conoce qué futbolistas tuvieron los mejores nú...,"Sin embargo, saliendo de los gustos de cada un..."
4,¿Yotún jugará en Universitario? Jean Ferrari: ...,"En el 2024, el jugador emblema de Sporting Cri...","Universitario ganó la Liga 1 este año, pero qu..."


In [56]:
df_football_news.to_csv("peruvian_football_new.csv", index=False, sep="|")

In [57]:
full_text_new = ''
for i in range(len(df_football_news)):
    full_text_new = full_text_new + df_football_news.loc[i, 'Title'] + ":" + "\n"
    full_text_new = full_text_new + df_football_news.loc[i, 'Text'] + "\n\n"

In [58]:
with open("peruvian_football_news.txt", "w") as f:
    f.write(full_text_new)

In [42]:
df_football_news_2

,Title,Subtitle,Text
0,Pide selección: Oliver Sonne anotó en la golea...,El lateral de la selección peruana se hizo pre...,"Atento Juan Reynoso. Oliver Sonne, lateral de ..."
1,"¡Así no, Leao! Butrón casi se fue a las manos ...",Conoce cuál habría sido el motivo que desató a...,Alianza Lima envió una delegación hacia Urugua...
2,¿Se pierde la primera final? Williams Riveros ...,El central de Universitario no entrenó a la pa...,Los entrenamientos de esta tarde en Universita...
3,El Kaiser está listo: Carlos Zambrano se recup...,Alianza Lima recibió una gran noticia de cara ...,Una buena noticia. Luego de que causara preocu...
4,¡Alianza va con todo! Barcos: Sería lindo ser ...,El goleador blanquiazul no se guardó nada y le...,Alianza Lima aún no define si comenzará como l...
...,...,...,...
242,Liga 1 en vilo a poco de su inicio: ocho clube...,"Representantes de Alianza Lima, Universitario,...",La Liga 1 y su inicio sigue en una gran incert...
243,José Bellina no descarta a Christian Cueva par...,El director deportivo del cuadro íntimo aclaró...,Alianza Lima presentó hoy a su plantel princip...
244,Empieza con triunfo: Alianza Lima venció por 2...,El conjunto íntimo enfrenta al club colombiano...,Minuto 90: ROJA para Homer Martínez por dura e...
245,Oficial: Universitario confirma el medio en do...,Los merengues hicieron oficial el medio de tra...,Queda todo listo. A solo horas de llevarse a c...


In [43]:
df_football_news_2.to_csv("peruvian_football_new_2.csv", index=False, sep="|")

In [44]:
full_text_new = ''
for i in range(len(df_football_news_2)):
    full_text_new = full_text_new + df_football_news_2.loc[i, 'Title'] + ":" + "\n"
    full_text_new = full_text_new + df_football_news_2.loc[i, 'Text'] + "\n\n"

In [45]:
with open("peruvian_football_news_2.txt", "w") as f:
    f.write(full_text_new)

In [6]:
df_football_news.head()

,Title,Subtitle,Text
0,Premio al esfuerzo: Piero Quispe fue elegido c...,El mediocampista de Universitario fue pieza fu...,"Luego de 10 años de sequía, Universitario form..."
1,A festejar crema: así fue la premiación de Uni...,"En el Monumental de Ate, los merengues serán p...",El equipo crema se dispone a dar la vuelta olí...
2,¿Se disculpó con Barcos? Flores: Tienen que ve...,Después de imitarlo en la celebración por el t...,En medio de las celebraciones por su nuevo tít...
3,Liga 1: ¿Quién fue el mejor jugador del fútbol...,Conoce qué futbolistas tuvieron los mejores nú...,"Sin embargo, saliendo de los gustos de cada un..."
4,¿Yotún jugará en Universitario? Jean Ferrari: ...,"En el 2024, el jugador emblema de Sporting Cri...","Universitario ganó la Liga 1 este año, pero qu..."


In [7]:
df_football_news_2.head()

,Title,Subtitle,Text
0,Pide selección: Oliver Sonne anotó en la golea...,El lateral de la selección peruana se hizo pre...,"Atento Juan Reynoso. Oliver Sonne, lateral de ..."
1,"¡Así no, Leao! Butrón casi se fue a las manos ...",Conoce cuál habría sido el motivo que desató a...,Alianza Lima envió una delegación hacia Urugua...
2,¿Se pierde la primera final? Williams Riveros ...,El central de Universitario no entrenó a la pa...,Los entrenamientos de esta tarde en Universita...
3,El Kaiser está listo: Carlos Zambrano se recup...,Alianza Lima recibió una gran noticia de cara ...,Una buena noticia. Luego de que causara preocu...
4,¡Alianza va con todo! Barcos: Sería lindo ser ...,El goleador blanquiazul no se guardó nada y le...,Alianza Lima aún no define si comenzará como l...


In [8]:
df_full_football_news = pd.concat([df_football_news, df_football_news_2], axis=0)
df_full_football_news

,Title,Subtitle,Text
0,Premio al esfuerzo: Piero Quispe fue elegido c...,El mediocampista de Universitario fue pieza fu...,"Luego de 10 años de sequía, Universitario form..."
1,A festejar crema: así fue la premiación de Uni...,"En el Monumental de Ate, los merengues serán p...",El equipo crema se dispone a dar la vuelta olí...
2,¿Se disculpó con Barcos? Flores: Tienen que ve...,Después de imitarlo en la celebración por el t...,En medio de las celebraciones por su nuevo tít...
3,Liga 1: ¿Quién fue el mejor jugador del fútbol...,Conoce qué futbolistas tuvieron los mejores nú...,"Sin embargo, saliendo de los gustos de cada un..."
4,¿Yotún jugará en Universitario? Jean Ferrari: ...,"En el 2024, el jugador emblema de Sporting Cri...","Universitario ganó la Liga 1 este año, pero qu..."
...,...,...,...
242,Liga 1 en vilo a poco de su inicio: ocho clube...,"Representantes de Alianza Lima, Universitario,...",La Liga 1 y su inicio sigue en una gran incert...
243,José Bellina no descarta a Christian Cueva par...,El director deportivo del cuadro íntimo aclaró...,Alianza Lima presentó hoy a su plantel princip...
244,Empieza con triunfo: Alianza Lima venció por 2...,El conjunto íntimo enfrenta al club colombiano...,Minuto 90: ROJA para Homer Martínez por dura e...
245,Oficial: Universitario confirma el medio en do...,Los merengues hicieron oficial el medio de tra...,Queda todo listo. A solo horas de llevarse a c...


In [12]:
df_full_football_news.reset_index(drop=True, inplace=True)
df_full_football_news.tail()

,Title,Subtitle,Text
292,Liga 1 en vilo a poco de su inicio: ocho clube...,"Representantes de Alianza Lima, Universitario,...",La Liga 1 y su inicio sigue en una gran incert...
293,José Bellina no descarta a Christian Cueva par...,El director deportivo del cuadro íntimo aclaró...,Alianza Lima presentó hoy a su plantel princip...
294,Empieza con triunfo: Alianza Lima venció por 2...,El conjunto íntimo enfrenta al club colombiano...,Minuto 90: ROJA para Homer Martínez por dura e...
295,Oficial: Universitario confirma el medio en do...,Los merengues hicieron oficial el medio de tra...,Queda todo listo. A solo horas de llevarse a c...
296,¿Por qué Alianza Lima rompió las negociaciones...,"Lo que parecía un fichaje inminente, de pronto...",Ahora no joven. Luego de que mucho se especula...


In [16]:
df_full_football_news['Title'] = df_full_football_news['Title'].apply(lambda x: x.replace("VIDEO", ""))
df_full_football_news['Subtitle'] = df_full_football_news['Subtitle'].apply(lambda x: x.replace("VIDEO", ""))
df_full_football_news['Text'] = df_full_football_news['Text'].apply(lambda x: x.replace("VIDEO", ""))

In [25]:
df_full_football_news['Title'] = df_full_football_news['Title'].apply(lambda x: x.replace(":", ""))
df_full_football_news['Subtitle'] = df_full_football_news['Subtitle'].apply(lambda x: x.replace(":", ""))
df_full_football_news['Text'] = df_full_football_news['Text'].apply(lambda x: x.replace(":", ""))

In [26]:
df_full_football_news.to_csv("full_peruvian_football_news.csv", index=False, sep="|")

In [27]:
full_text_new = ''
for i in range(len(df_full_football_news)):
    full_text_new = full_text_new + df_full_football_news.loc[i, 'Title'] + ":" + "\n"
    full_text_new = full_text_new + df_full_football_news.loc[i, 'Text'] + "\n\n"

In [28]:
with open("full_peruvian_football_news.txt", "w") as f:
    f.write(full_text_new)